In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sakura-uta-voice/sakura-uta-voice/mkd_003210.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/knd_000844.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/aid_001068.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/hrd_002725.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/szd_001113.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/aid_000196.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/mkd_001585.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/hrd_001337.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/szd_000039.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/hrd_002483.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/mzd_000258.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/skrd_000212.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/ymd_002254.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/rnd_000959.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/szd_001139.ogg
/kaggle/input/sakura-uta-voice/sakura-uta-voice/sud_00

fine tune

In [5]:
import torchaudio
from pathlib import Path

train_dir = Path("/kaggle/input/sakura-2/")

waveforms = []
for wav_path in train_dir.glob("*.wav"):
    waveform, sr = torchaudio.load(wav_path)
    waveforms.append(waveform)


In [6]:
from huggingface_hub import snapshot_download

# โหลด model ทั้งหมดมาไว้ใน local (เช่น /kaggle/working/rvc_model)
local_dir = snapshot_download(repo_id="sail-rvc/Hatsune_Miku__RVC_v2_", local_dir="/kaggle/working/rvc_model")
print("Model path:", local_dir)


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Model path: /kaggle/working/rvc_model


In [7]:
local_dir = snapshot_download(repo_id="sail-rvc/Hatsune_Miku__RVC_v2_", local_dir="/kaggle/working/rvc_model")
print("Model path:", local_dir)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Model path: /kaggle/working/rvc_model


In [8]:
# รันในเซลล์แรกของ notebook
!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git
!mv Retrieval-based-Voice-Conversion-WebUI /kaggle/working/
!pip install -r /kaggle/working/Retrieval-based-Voice-Conversion-WebUI/requirements.txt
!apt-get install -y ffmpeg
!pip install pyworld openvcrepe fairseq

Cloning into 'Retrieval-based-Voice-Conversion-WebUI'...
remote: Enumerating objects: 4593, done.
remote: Total 4593 (delta 0), reused 0 (delta 0), pack-reused 4593 (from 1)
Receiving objects: 100% (4593/4593), 14.33 MiB | 40.10 MiB/s, done.
Resolving deltas: 100% (2884/2884), done.
mv: 'Retrieval-based-Voice-Conversion-WebUI' and '/kaggle/working/Retrieval-based-Voice-Conversion-WebUI' are the same file
Ignoring onnxruntime: markers 'sys_platform == "darwin"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.6 MB/s eta 0:00:00a 0:00:01
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with 

In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import librosa
import soundfile as sf
from pathlib import Path

# ตรวจสอบ GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
import torch

# โหลดโมเดลและตรวจสอบโครงสร้าง
model_path = "/kaggle/working/rvc_model/model.pth"
base_model = torch.load(model_path, map_location='cpu')

# ตรวจสอบประเภทข้อมูล
print(f"ประเภทของโมเดล: {type(base_model)}")

# ถ้าเป็น Dictionary ให้ดู keys ทั้งหมด
if isinstance(base_model, dict):
    print("\nKeys ในโมเดล:")
    for key in base_model.keys():
        print(f"- {key}")
        # ตรวจสอบรายละเอียดเพิ่มเติมถ้าเป็น state_dict
        if isinstance(base_model[key], dict) and 'weight' in next(iter(base_model[key].keys()), None):
            print(f"  (มี state_dict ย่อย)")

# ถ้าเป็น OrderedDict ให้ดู keys ตัวอย่าง
elif hasattr(base_model, 'keys'):
    print("\nตัวอย่าง keys ใน state_dict:")
    for i, key in enumerate(base_model.keys()):
        if i < 5:  # แสดงแค่ 5 keys แรก
            print(f"- {key}")
        if i == 5:
            print("...")

ประเภทของโมเดล: <class 'collections.OrderedDict'>

Keys ในโมเดล:
- weight
  (มี state_dict ย่อย)
- config
- info
- sr
- f0
- version


In [11]:
!pip install -r /kaggle/working/Retrieval-based-Voice-Conversion-WebUI/requirements.txt

Ignoring onnxruntime: markers 'sys_platform == "darwin"' don't match your environment
  Using cached aria2-0.0.1b0-py3-none-manylinux_2_17_x86_64.whl.metadata (28 kB)
  Using cached numba-0.56.4.tar.gz (2.4 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [12]:
!python /kaggle/working/Retrieval-based-Voice-Conversion-WebUI/configs/config.py

In [14]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import librosa
import soundfile as sf
from pathlib import Path

# ตรวจสอบ GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# โหลดโมเดล
model_path = "/kaggle/working/rvc_model/model.pth"
base_model = torch.load(model_path, map_location=device)

# ตรวจสอบโครงสร้างโมเดล
if isinstance(base_model, nn.Module):
    # ถ้าโหลดมาเป็นโมเดลโดยตรง
    model = base_model.to(device)
elif 'model' in base_model:
    # กรณีใช้ใน RVC framework
    model = base_model['model'].to(device)
elif 'state_dict' in base_model:
    # กรณีทั่วไป
    model = RVCModel().to(device)
    model.load_state_dict(base_model['state_dict'])
else:
    # กรณีเป็น state_dict โดยตรง
    try:
        model = RVCModel().to(device)
        model.load_state_dict(base_model)
    except:
        # สร้างโมเดลใหม่ถ้าโหลดไม่สำเร็จ
        print("⚠️ โหลดโมเดลไม่สำเร็จ สร้างโมเดลใหม่")
        # model = RVCModel().to(device)

# สถาปัตยกรรมโมเดล (ปรับตามโครงสร้างจริง)
class RVCModel(nn.Module):
    def __init__(self, in_dim=128, out_dim=128, hidden_dim=256):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(in_dim, hidden_dim, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.InstanceNorm1d(hidden_dim)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(hidden_dim, out_dim, kernel_size=5, padding=2),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.encoder(x)
        return self.decoder(x)

⚠️ โหลดโมเดลไม่สำเร็จ สร้างโมเดลใหม่


In [15]:
# แช่แข็งเลเยอร์ที่ไม่ต้องการอัปเดต
for name, param in model.named_parameters():
    if 'encoder' in name:  # แช่แข็ง encoder
        param.requires_grad = False
    else:
        param.requires_grad = True

# เตรียมข้อมูล
class VoiceDataset(Dataset):
    def __init__(self, data_dir, sr=22050, segment_length=22050):
        self.files = list(Path(data_dir).rglob("*.wav"))
        self.sr = sr
        self.segment_length = segment_length
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        try:
            audio, sr = librosa.load(self.files[idx], sr=self.sr, duration=2.0)  # จำกัดเวลา 2 วินาที
            if len(audio) < self.segment_length:
                audio = np.pad(audio, (0, self.segment_length - len(audio)))
            else:
                audio = audio[:self.segment_length]
            
            # Mel-spectrogram
            mel = librosa.feature.melspectrogram(
                y=audio, sr=sr, n_fft=1024, hop_length=256, n_mels=128)
            return torch.tensor(mel, dtype=torch.float32)
        except Exception as e:
            print(f"Error loading {self.files[idx]}: {str(e)}")
            return torch.zeros((128, 86))  # ค่าขนาดมาตรฐาน

# ตั้งค่า DataLoader
train_set = VoiceDataset("/kaggle/input/sakura-2/")
valid_set = VoiceDataset("/kaggle/input/validation-anime/validation/")

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_set, batch_size=16, num_workers=2)

# การเทรน
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

for epoch in range(100):
    model.train()
    train_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        outputs = model(batch)
        loss = nn.MSELoss()(outputs, batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in valid_loader:
            batch = batch.to(device)
            outputs = model(batch)
            val_loss += nn.MSELoss()(outputs, batch).item()
    
    avg_train = train_loss / len(train_loader)
    avg_val = val_loss / len(valid_loader)
    
    print(f"Epoch {epoch+1:03d} | Train: {avg_train:.4f} | Val: {avg_val:.4f}")
    scheduler.step(avg_val)
    
    # บันทึกโมเดลทุก 10 epoch
    if (epoch + 1) % 10 == 0:
        torch.save(model.state_dict(), f"model_epoch_{epoch+1}.pth")

NameError: name 'model' is not defined

In [ ]:
def convert_voice(input_path, output_path, model):
    # โหลดเสียง
    audio, sr = librosa.load(input_path, sr=22050)
    
    # แยกเป็นช่วงสั้นๆ
    segments = []
    segment_length = 22050  # 1 วินาที
    for i in range(0, len(audio), segment_length):
        seg = audio[i:i+segment_length]
        if len(seg) < segment_length:
            seg = np.pad(seg, (0, segment_length - len(seg)))
        segments.append(seg)
    
    # แปลงแต่ละส่วน
    converted_segments = []
    for seg in segments:
        mel = librosa.feature.melspectrogram(
            y=seg, sr=sr, n_fft=1024, hop_length=256, n_mels=128)
        mel_tensor = torch.tensor(mel).unsqueeze(0).to(device)
        
        with torch.no_grad():
            converted_mel = model(mel_tensor).cpu().numpy().squeeze()
        
        # Griffin-Lim reconstruction
        audio_seg = librosa.feature.inverse.mel_to_audio(
            converted_mel, sr=sr, n_fft=1024, hop_length=256)
        converted_segments.append(audio_seg)
    
    # รวมส่วนและบันทึก
    full_audio = np.concatenate(converted_segments)
    sf.write(output_path, full_audio, sr)

# ทดสอบ
model.eval()
convert_voice("/kaggle/input/datasets-input-voice/input_25_vocal.wav", "/kaggle/working/converted.wav", model)